In [167]:
import regex as re
import time
from datetime import datetime
import requests
import pandas as pd
from urllib.parse import urlparse
from urllib.parse import parse_qs
from bs4 import BeautifulSoup
import json, re
from selenium import webdriver
from selenium.webdriver.common.by import By
from google_play_scraper import app
!pip install langdetect
from langdetect import detect

You should consider upgrading via the 'c:\users\shallom\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=63f066f9249982e59ce8803e130767bcf1ad50abcdf7f9456617151fd0bc2a2b
  Stored in directory: c:\users\shallom\appdata\local\pip\cache\wheels\c5\96\8a\f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [ ]:
country_cd = []
code = []
req = requests.get('https://www.mobileaction.co/top-apps/travel-and-local-22/android/us')
all = BeautifulSoup(req.content, 'html.parser')
v = all.find_all('i', class_='flag-icon')
for b in v:
    pro = str(b.parent.get_text()).replace(' ', '')
    country_cd.append(pro.strip('\n'))

In [ ]:
iso = pd.read_csv('datasets/iso_code.csv')
c_code = []
for country in country_cd:
    c_code.append([f for f in iso.loc[iso['Name'] == re.sub(r"(\w)([A-Z])", r"\1 \2", country)].Code.values])

In [ ]:
pd.set_option('display.max_rows', 10)
out = [x for x in c_code]
f_code = pd.DataFrame(out, columns=['Code']).dropna()

In [ ]:
vc = [d for d in f_code.Code]
# data_d = pd.DataFrame(columns=['CountryISO', 'AppName', 'AppID', 'Usage', 'Platform'])

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
}
for c in vc[56:]:
    req = requests.get(f'https://www.mobileaction.co/top-apps/travel-and-local-22/android/{c}', headers=headers, stream=True)
    all = BeautifulSoup(req.content, 'html.parser')

    # Free
    res = all.find_all('td', class_='free-col')
    for x in res:
        cvv = BeautifulSoup(str(x), 'html.parser')
        try:
            pro = str(cvv.find('a', class_='app-name').get_text()).replace(' ', '')
            data_d = data_d.append({'CountryISO': c, 'AppName': pro.strip('\n'), 'Usage': 'Free', 'Platform': 'Android'}, ignore_index=True)
        except:
            continue

    # paid
    res = all.find_all('td', class_='paid-col')
    for x in res:
        cvv = BeautifulSoup(str(x), 'html.parser')
        try:
            pro = str(cvv.find('a', class_='app-name').get_text()).replace(' ', '')
            data_d = data_d.append({'CountryISO': c, 'AppName': pro.strip('\n'), 'Usage': 'Paid', 'Platform': 'Android'}, ignore_index=True)
        except:
            continue

In [12]:
data_d = pd.read_csv('android_apps.csv')
data_d.drop(columns=['Unnamed: 0'], inplace=True)
data_d.insert(3, 'AppRating', None)
data_d

,CountryISO,AppName,AppID,AppRating,Usage,Platform
0,US,Upside-Cashbackongas&food,NaN,None,Free,Android
1,US,Booking.com:Hotelsandmore,NaN,None,Free,Android
2,US,"Expedia:Hotels,Flights&Car",NaN,None,Free,Android
3,US,Airbnb,NaN,None,Free,Android
4,US,VrboVacationRentals,NaN,None,Free,Android
...,...,...,...,...,...,...
3975,TN,SchengenVisaApp‏,NaN,None,Free,Android
3976,TN,OffroadPickupCargoTruck3D‏,NaN,None,Free,Android
3977,TN,GPSOfflineMaps&Navigation‏,NaN,None,Free,Android
3978,TN,E-Taxi‏,NaN,None,Free,Android


## Scraping app IDs

In [28]:
def _scrapy(
    query: str, country: str, filter_by: str = "apps"
):
    params = {
        "q": query,
        "gl": country,
        "c": filter_by
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.79 Safari/537.36",
    }

    html = requests.get("https://play.google.com/store/search", params=params, headers=headers, timeout=30)
    soup = BeautifulSoup(html.text, "html.parser")

    try:
        try:
            app = soup.select_one("[jscontroller=tKHFxf]")
            title = app.select_one(".DdYX5").text
            # company = app.select_one(".wMUdtb").text
            app_id = app.select_one("a")["href"].split("id=")[1]
            try:
                rating = re.search(r"\d{1}\.\d{1}", app.select_one(".ubGTjb div")["aria-label"]).group()
            except:
                rating = None    
        except:
            try:
                s_html = requests.get(f'https://play.google.com{soup.select_one(".Qfxief")["href"]}')
                m_soup = BeautifulSoup(s_html.content, "html.parser")
                title = m_soup.select_one('.Fd93Bb').findChild('span').text
                parsed_url = urlparse(s_html.url)
                app_id = parse_qs(parsed_url.query)['id'][0]
                try:
                    rating = re.search(r"\d{1}\.\d{1}", m_soup.select_one(".TT9eCd")["aria-label"]).group()
                except:
                    rating = None
            except:
                s_html = requests.get(f'https://play.google.com{soup.select_one(".Si6A0c.Gy4nib")["href"]}')
                m_soup = BeautifulSoup(s_html.content, "html.parser")
                title = m_soup.select_one('.Fd93Bb').findChild('span').text
                parsed_url = urlparse(s_html.url)
                app_id = parse_qs(parsed_url.query)['id'][0]
                try:
                    rating = re.search(r"\d{1}\.\d{1}", m_soup.select_one(".TT9eCd")["aria-label"]).group()
                    print(rating)
                except:
                    rating = None
    except:
        app_id = None
        title = None
        rating = None

    data_d.loc[data_d.AppName == query, 'AppID'] = app_id
    data_d.loc[data_d.AppName == query, 'AppRating'] = float(rating) if rating else rating
    data_d.loc[data_d.AppName == query, 'AppName'] = title

# data_d = data_d.reset_index()

In [ ]:
for ind, nm in data_d[1791:].iterrows():
    _scrapy(query=re.sub(r"(\w)([A-Z])", r"\1 \2", str(nm.AppName)), country=nm.CountryISO, filter_by="apps")
    data_d.to_csv('app_d.csv')
    print(ind)

In [11]:
gb = pd.read_csv('app_d.csv')
gb.dropna(subset=['AppID'], inplace=True)
gb.reset_index(inplace=True)
gb.drop(columns=['Unnamed: 0', 'index'], inplace=True)
gb.insert(4, 'Ratings', None)
gb.insert(5, 'Reviews', None)
gb.insert(6, 'Downloads', None)
gb.insert(7, 'Developer', None)
gb.insert(8, 'Summary', None)
gb.insert(9, 'inAppPurchase', None)
gb.insert(10, 'Currency', None)
gb.insert(11, 'Price', None)
gb.insert(12, 'Category', None)
gb.insert(13, 'Contains Ads', None)
gb.insert(14, 'releaseDate', None)
gb.insert(15, 'currentVersion', None)
gb.insert(16, 'lastUpdated', None)
gb.insert(17, 'appURL', None)
gb.insert(18, 'website', None)
gb.insert(19, 'histogram', None)
gb

,CountryISO,AppName,AppID,AppRating,Ratings,Reviews,Downloads,Developer,Summary,inAppPurchase,...,Category,Contains Ads,releaseDate,currentVersion,lastUpdated,appURL,website,histogram,Usage,Platform
0,US,Upside-Cash back on gas & food,com.upside.consumer.android,4.4,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Free,Android
1,US,Booking.com: Hotels and more,com.booking,4.6,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Free,Android
2,US,"Expedia: Hotels, Flights & Car",com.expedia.bookings,3.7,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Free,Android
3,US,Airbnb,com.airbnb.android,4.5,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Free,Android
4,US,"Priceline: Hotel, Flight & Car",com.priceline.android.negotiator,4.7,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Free,Android
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,TN,Nouvelair,tn.keyrus.nouvelair,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Free,Android
1129,TN,وقتاش الكار - أوقات حافلات تونس,com.maher.tunisbus,3.1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Free,Android
1130,TN,Perfect Compass (with weather),pl.netigen.compass,4.1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Free,Android
1131,TN,Partagi.tn,com.capsinet.partagitn,3.6,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Free,Android


In [158]:
# for ind, id in gb.iterrows():
result = app(
    'com.bangkokairways.main',
    lang='en',
    country='de'
)

gb.loc[gb.AppID == 'com.bangkokairways.main', 'Ratings'] = result['ratings']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'Reviews'] = result['reviews']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'Downloads'] = result['installs']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'Developer'] = result['developer']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'Summary'] = result['summary']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'inAppPurchase'] = result['inAppProductPrice']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'Currency'] = result['currency']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'Price'] = result['price']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'Category'] = result['genre']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'Contains Ads'] = result['containsAds']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'releaseDate'] = result['released']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'currentVersion'] = result['version']
try:
    gb.loc[gb.AppID == 'com.bangkokairways.main', 'lastUpdated'] = datetime.fromtimestamp(result['updated']).strftime('%d-%m-%Y %H:%M:%S')
except:
    gb.loc[gb.AppID == 'com.bangkokairways.main', 'lastUpdated'] = 'Unknown'
gb.loc[gb.AppID == 'com.bangkokairways.main', 'histogram'] = str(result['histogram'])
gb.loc[gb.AppID == 'com.bangkokairways.main', 'appURL'] = result['url']
gb.loc[gb.AppID == 'com.bangkokairways.main', 'website'] = result['developerWebsite']
gb.to_csv('zxcbv.csv')

In [163]:
gb.dropna(subset=['Downloads'], inplace=True)
gb.reset_index(inplace=True)
gb.drop(columns=['index'], inplace=True)
gb

,CountryISO,AppName,AppID,AppRating,Ratings,Reviews,Downloads,Developer,Summary,inAppPurchase,...,Category,Contains Ads,releaseDate,currentVersion,lastUpdated,appURL,website,histogram,Usage,Platform
0,US,Upside-Cash back on gas & food,com.upside.consumer.android,4.4,166676,85411,"5,000,000+",Upside Services Inc.,"Earn cash back every day at gas stations, groc...",None,...,Travel & Local,False,"Apr 24, 2016",5.5,16-06-2022 15:40:02,https://play.google.com/store/apps/details?id=...,https://www.upside.com,"[14400, 4282, 5567, 22235, 120187]",Free,Android
1,US,Booking.com: Hotels and more,com.booking,4.6,3418793,74194,"500,000,000+",Booking.com Hotels & Vacation Rentals,Book your whole trip in one app.,None,...,Travel & Local,False,"Feb 4, 2011",Varies with device,07-07-2022 15:32:56,https://play.google.com/store/apps/details?id=...,http://www.booking.com/apps.html?source=google...,"[172428, 37598, 93014, 454433, 2661285]",Free,Android
2,US,"Expedia: Hotels, Flights & Car",com.expedia.bookings,3.7,279939,47095,"10,000,000+",Expedia,"Book flight, car or packages. Get instant acce...",None,...,Travel & Local,True,"May 10, 2011",22.26.0,07-07-2022 17:05:19,https://play.google.com/store/apps/details?id=...,https://www.expedia.com/service/,"[67923, 10781, 11857, 25869, 163500]",Free,Android
3,US,Airbnb,com.airbnb.android,4.5,1315518,1151,"100,000,000+",Airbnb,The Airbnb app brings together a global commun...,None,...,Travel & Local,False,"Jan 17, 2012",Varies with device,30-06-2022 08:12:28,https://play.google.com/store/apps/details?id=...,https://www.airbnb.com/mobile,"[115138, 6649, 69643, 198780, 924607]",Free,Android
4,US,"Priceline: Hotel, Flight & Car",com.priceline.android.negotiator,4.7,249909,44413,"10,000,000+",priceline.com,Save on your next trip with exclusive deals on...,None,...,Travel & Local,False,"Nov 10, 2010",6.5.245,08-07-2022 13:53:25,https://play.google.com/store/apps/details?id=...,http://www.priceline.com,"[5496, 2174, 9752, 31969, 200513]",Free,Android
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,TN,Nouvelair,tn.keyrus.nouvelair,NaN,420,62,"100,000+","Nouvelair, Tunisie",The Nouvelair app helps you discover its desti...,None,...,Travel & Local,False,"Jun 29, 2017",5.1.3.36.1,14-03-2022 16:11:20,https://play.google.com/store/apps/details?id=...,https://www.nouvelair.com,"[106, 11, 40, 40, 212]",Free,Android
1125,TN,وقتاش الكار - أوقات حافلات تونس,com.maher.tunisbus,3.1,19,5,"1,000+",BricoMaher,This application enables to know bus times in ...,None,...,Travel & Local,True,"Mar 18, 2021",5.0,11-04-2021 00:16:02,https://play.google.com/store/apps/details?id=...,https://x903c9200.app-ads-txt.com/app-ads.txt,"[7, 0, 2, 2, 7]",Free,Android
1126,TN,Perfect Compass (with weather),pl.netigen.compass,4.1,22472,33,"5,000,000+",Netigen Tools,Beautiful and precise compass app with weather...,None,...,Tools,True,"May 20, 2014",Varies with device,24-06-2022 12:49:21,https://play.google.com/store/apps/details?id=...,http://tools.netigen.pl/,"[3121, 124, 1997, 2247, 14731]",Free,Android
1127,TN,Partagi.tn,com.capsinet.partagitn,3.6,143,50,"10,000+",Partagi.tn,This is the FREE & colocation services carpool...,None,...,Travel & Local,False,"Jun 18, 2017",2.2.2,12-11-2019 10:38:13,https://play.google.com/store/apps/details?id=...,https://www.partagi.tn,"[37, 2, 10, 10, 77]",Free,Android


In [200]:
gb.AppName = gb.AppName.str.encode("ascii", errors="ignore").str.decode(encoding='utf-8').str.replace('-', ' ')
gb

,CountryISO,AppName,AppID,AppRating,Ratings,Reviews,Downloads,Developer,Summary,inAppPurchase,...,Contains Ads,releaseDate,currentVersion,lastUpdated,appURL,website,histogram,Usage,Platform,lang
0,US,Upside Cash back on gas & food,com.upside.consumer.android,4.4,166676,85411,"5,000,000+",Upside Services Inc.,"Earn cash back every day at gas stations, groc...",None,...,False,"Apr 24, 2016",5.5,16-06-2022 15:40:02,https://play.google.com/store/apps/details?id=...,https://www.upside.com,"[14400, 4282, 5567, 22235, 120187]",Free,Android,en
1,US,Booking.com: Hotels and more,com.booking,4.6,3418793,74194,"500,000,000+",Booking.com Hotels & Vacation Rentals,Book your whole trip in one app.,None,...,False,"Feb 4, 2011",Varies with device,07-07-2022 15:32:56,https://play.google.com/store/apps/details?id=...,http://www.booking.com/apps.html?source=google...,"[172428, 37598, 93014, 454433, 2661285]",Free,Android,en
2,US,"Expedia: Hotels, Flights & Car",com.expedia.bookings,3.7,279939,47095,"10,000,000+",Expedia,"Book flight, car or packages. Get instant acce...",None,...,True,"May 10, 2011",22.26.0,07-07-2022 17:05:19,https://play.google.com/store/apps/details?id=...,https://www.expedia.com/service/,"[67923, 10781, 11857, 25869, 163500]",Free,Android,en
3,US,Airbnb,com.airbnb.android,4.5,1315518,1151,"100,000,000+",Airbnb,The Airbnb app brings together a global commun...,None,...,False,"Jan 17, 2012",Varies with device,30-06-2022 08:12:28,https://play.google.com/store/apps/details?id=...,https://www.airbnb.com/mobile,"[115138, 6649, 69643, 198780, 924607]",Free,Android,lt
4,US,"Priceline: Hotel, Flight & Car",com.priceline.android.negotiator,4.7,249909,44413,"10,000,000+",priceline.com,Save on your next trip with exclusive deals on...,None,...,False,"Nov 10, 2010",6.5.245,08-07-2022 13:53:25,https://play.google.com/store/apps/details?id=...,http://www.priceline.com,"[5496, 2174, 9752, 31969, 200513]",Free,Android,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,TN,Nouvelair,tn.keyrus.nouvelair,NaN,420,62,"100,000+","Nouvelair, Tunisie",The Nouvelair app helps you discover its desti...,None,...,False,"Jun 29, 2017",5.1.3.36.1,14-03-2022 16:11:20,https://play.google.com/store/apps/details?id=...,https://www.nouvelair.com,"[106, 11, 40, 40, 212]",Free,Android,fr
1125,TN,,com.maher.tunisbus,3.1,19,5,"1,000+",BricoMaher,This application enables to know bus times in ...,None,...,True,"Mar 18, 2021",5.0,11-04-2021 00:16:02,https://play.google.com/store/apps/details?id=...,https://x903c9200.app-ads-txt.com/app-ads.txt,"[7, 0, 2, 2, 7]",Free,Android,ar
1126,TN,Perfect Compass (with weather),pl.netigen.compass,4.1,22472,33,"5,000,000+",Netigen Tools,Beautiful and precise compass app with weather...,None,...,True,"May 20, 2014",Varies with device,24-06-2022 12:49:21,https://play.google.com/store/apps/details?id=...,http://tools.netigen.pl/,"[3121, 124, 1997, 2247, 14731]",Free,Android,en
1127,TN,Partagi.tn,com.capsinet.partagitn,3.6,143,50,"10,000+",Partagi.tn,This is the FREE & colocation services carpool...,None,...,False,"Jun 18, 2017",2.2.2,12-11-2019 10:38:13,https://play.google.com/store/apps/details?id=...,https://www.partagi.tn,"[37, 2, 10, 10, 77]",Free,Android,id


In [ ]:
gb.to_csv('zxcbv.csv')